In [4]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [30]:
def create_model(num_classes, input_shape, type_extractor = 'vgg') -> tf.keras.Model:
  if type_extractor == 'vgg':
    feature_extractor = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
  elif type_extractor == 'inception':
    feature_extractor = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
  elif type_extractor == 'resnet':
    feature_extractor = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
  else:
    raise ValueError('type_extractor must be vgg, inception or resnet')
  
  model = tf.keras.Sequential()
  model.add(feature_extractor)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=4096,activation="relu", input_dim=1000))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(units=4096,activation="relu"))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
  return model

In [31]:
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import preprocess_input
import numpy as np

image = load_img('./images/db1.jpg', target_size=(224, 224))
image = np.array(image)
images = np.array([image])
images = preprocess_input(images)

In [33]:
#multi modal loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model = create_model(num_classes=2, input_shape=images[0].shape, type_extractor='vgg')
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])